In [ ]:
import pandas as pd
import random 
from datetime import datetime,timedelta
import string
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import qrcode

df_station = pd.read_csv(r"C:\Users\lenovo\Desktop\jupyter\STATIONNAME.csv", encoding='latin-1')
df = pd.DataFrame(columns=['UTS NO.', 'mobile_code', 'Source Station', 'Destination', 'Date & Time', 'Distance', 'SAC', 'Adult', 'Children', 'Fare', 'IR', 'Reserve NO'])
df_pltf = pd.DataFrame(columns=['UTS NO.', 'Station', 'Date','Time', 'SAC','No. of person','Fare', 'IR', 'Reserve NO'])
df_mst = pd.DataFrame(columns=['UTS NO.', 'mobile_code', 'Source Station', 'Destination', 'Distance', 'SAC', 'Passenger', 'Fare', 'IR', 'Reserve NO','Age','Name','ID','End Date'])

# this funcation is given the distince between source and destionatio
def calculate_distance(source, destination):
    source_coords = get_coordinates(source)
    dest_coords = get_coordinates(destination)

    if source_coords and dest_coords:
        distance = geodesic(source_coords, dest_coords).kilometers
        return distance
    else:
        return None

def get_coordinates(location_name):
    geolocator = Nominatim(user_agent="railway_distance_calculator")
    location = geolocator.geocode(location_name)
    if location:
        return location.latitude, location.longitude
    else:
        print(f"Coordinates not found for {location_name}")
        return None

# this function is generate the qr code 
def qr_code(d):
    qr = qrcode.QRCode(
    version=1,
    error_correction=qrcode.constants.ERROR_CORRECT_L,
    box_size=4,
    border=4,
    )
    qr.add_data(d)
    qr.make(fit=True)
    
    # Create an image from the QR Code instance
    img = qr.make_image(fill_color="black", back_color="white")
    img.show()


#this funcation is generted the IR code
def generate_random_code():
    # Define the format of the code
    st = random.randint(10,99)
    prefix = "AAAGM"
    numbers = ''.join(random.choices(string.digits, k=4))
    suffix = ''.join(random.choices(string.ascii_uppercase, k=2))
    current_time = datetime.now()
    formatted_date = current_time.strftime('%Y')
    year = formatted_date[2:]

    # Combine the parts to form the code
    random_code = f"{st}{prefix}{numbers}CA{year}{suffix}"

    return random_code


#this number is generated string code
def generate_random_number():
    # Define the format of the number
    prefix = "R"
    numbers = ''.join(random.choices('0123456789', k=5))

    # Combine the parts to form the number
    random_number = f"{prefix}{numbers}"

    return random_number



#this funcation is generted the IR code
def generate_ir_code():
    # Define the format of the code
    st = random.randint(10,99)
    prefix = "XS"
    numbers = ''.join(random.choices(string.digits, k=4))
    suffix = random.randint(1001,9999)
    current_time = datetime.now()
    formatted_date = current_time.strftime('%Y')
    year = formatted_date[2:]

    # Combine the parts to form the code
    random_code = f"{prefix}{st}C{year}{suffix}F"

    return random_code


# this funcatio is station
def suggest_places(partial_word,df):
    filtered_df = df[df['Station name'].str.contains(partial_word.title()) | df['Station code'].str.contains(partial_word.upper())]
    return list(zip(filtered_df['Station name'], filtered_df['Station code']))

# this give suggest station name
def suggest_city():
    print('Enter The Station Name or Code')
    source = input('From: ')
    destination = input('To: ')
    if type(source) == str:
        suggested_sources = suggest_places(source, df_station)
        if suggested_sources:
            for place in suggested_sources:
                print(f"Source Station name: {place[0]}, Source Station code: {place[1]}")
        else:
            print(f"No suggestions found for '{source}'")
    if type(destination) == str:
        suggested_destinations = suggest_places(destination,df_station)
        if suggested_destinations:
            for place in suggested_destinations:
                print(f"Destination Station name: {place[0]}, Destination Station code: {place[1]}")
        else:
            print(f"No suggestions found for '{destination}'")
# suggest_city()

# this funcation is calculate the ticket fare according to distance than pan
def fare(km,adult,children):
    km = round(km)
    if km  <= 80 :
        km = round(km,2)
        p = km/10
        total = p*7.2
        person = total * adult
        non_adult = total * children / 2
        total_price = person + non_adult
        return total_price
    elif km > 80 :
        km = round(km,2)
        p = km/10
        total = p*3
        person = total * adult
        non_adult = total * children / 2
        total_price = person + non_adult
        return total_price

def mst_prce_ticket(km):
    if 1<= km  <= 20:
        return 100
    elif 21<= km <= 45:
        return 185
    elif 46<= km <= 70:
        return 270
    elif 71<= km <= 100:
        return 355
    elif 101<= km <= 135:
        return 440
    elif 136<= km <=165 :
        return 525
    elif 166<= km <= 205:
        return 615
    elif 206<= km <= 235:
        return 700
    elif 236<= km <= 275:
        return 785
    elif 756<= km <= 300:
        return 870
    elif 301<= km <=360 :
        return 955
    elif 361<= km <= 390:
        return 1040
    elif 391<= km <= 420:
        return 1125
    elif 421<= km <= 470:
        return 1215
    elif 471<= km <=490 :
        return 1300
    elif 491<= km <=500 :
        return 1385
    else: 
        return (km * 5)

    
# mst_prce_ticket()    

# this function is book ticket for mst
def booking_mst():
    try:
        global df_mst
        t_s = []
        mobile  = []
        s_st = []
        d_st = []
        d_e = []
        T_e = []
        Dis = []
        s_c = []
        Pss = []
        T_f = []
        i_r = []
        resv_code = []
        a_ge = []
        na = []
        id_n = []
        End = []
        suggest_city()
        print('Enter The Station Name or Code')
        source = input('From: ')
        destination = input('To: ')
        if any(df_station['Station name'].isin([source.title()])) or any(df_station['Station code'].isin([source.upper()])) \
                and any(df_station['Station name'].isin([destination.title()])) or any(df_station['Station code'].isin([destination.upper()])):
            
            source_name = df_station[(df_station['Station name'].isin([source.title()]) | df_station['Station code'].isin([source.upper()]))]['Station name'].values[0]
            destination_name = df_station[(df_station['Station name'].isin([destination.title()]) | df_station['Station code'].isin([destination.upper()]))]['Station name'].values[0]
    
            distance_km = calculate_distance(source_name, destination_name)
    
            if distance_km is not None:
                # print(f"The distance between {source_name} and {destination_name} is {distance_km:.2f} km.")
                Ticket_no = generate_ir_code()
                mo_code = random.randint(100000000000,999999999999)
                sac = random.randint(111111,999999)
                random_code = generate_random_code()
                random_number = generate_random_number()
                name = (input('Enter Your Name: ')).title()
                age = int(input('Enter Your Age:'))
                id = int(input('Enter Your ID No: '))
                current_time = datetime.now()
                price = mst_prce_ticket(distance_km)
                formatted_time = current_time.strftime("%H:%M:%S")
                formatted_date = current_time.strftime('%d/%m/%Y')
                # this give me next 30 days date from current date
                day_nex = current_time + timedelta(days=30)
                End_date = day_nex.strftime('%d/%m/%Y')
                km_in = round(distance_km)
                t_s.append(Ticket_no)
                mobile.append(mo_code)
                s_st.append(source_name)
                d_st.append(destination_name)
                d_e.append(formatted_date)
                T_e.append(formatted_time)
                Dis.append(km_in)
                s_c.append(sac)
                Pss.append(name)
                T_f.append(price)
                i_r.append(random_code)
                resv_code.append(random_number)
                a_ge.append(age)
                na.append(name)
                id_n.append(id)
                End.append(End_date)
                mst_data = pd.DataFrame({'UTS NO.':t_s,
                             'mobile_code':mobile,
                             'Source Station':s_st,
                             'Destination':d_st,
                             'Date':d_e,
                             'Time':T_e,
                             'Distance': Dis,
                             'SAC': s_c,
                             'Passenger':Pss,
                             'Fare':T_f,
                             'End Date': End,
                             'IR':i_r,
                             'Reserve NO':resv_code,
                             'Name':na,
                             'Age':a_ge,
                             'ID':id_n,
                                }) 
    
                df_mst = pd.concat([df_mst, mst_data], ignore_index=True)
                print(f'''-------------------------------Ticket------------------------------
HAPPY JOURNEY                               MONTHLY
-------------------------------------------------------------------
                              ADULT SEASON          {formatted_date}
₹Rs {price}/-                                            {mo_code} 
UTS NO:{Ticket_no}                                       MONTHLY
ID Card Number: {id}
Passenger Name: {name}
Age:{age}Years                                        Between
-------------------------------------------------------------------
(S) {source_name}.                                                    
(D) {destination_name}.                                                                                     
CLASS: SECOND(||)                          TRAIN TYPE: ORDINARY(O)
-------------------------------------------------------------------
(via)---
-------------------------------------------------------------------
SAC:{sac}                         IR:{random_code}                          
Validity:FROM:{formatted_date} TO {End_date}
{random_number}                    Distance:{round(distance_km)} Km.
Booking Time:{formatted_date} {formatted_time}
''')
                
            else:
                print("Distance calculation failed.")
        else:
            print('Station Not Found:')
    except Exception as e:
            print(f"An error occurred: {e}")
            print("Please retry.")
# booking_mst()

# this function on ticket  normal booking
def booking():
    try:
        global df
        suggest_city()
        print('Enter The Station Name or Code')
        source = input('From: ')
        destination = input('To: ')
        uts_no = []
        mobile_code = []
        from_station = []
        to_station = []
        datetm = []
        dis_in_km = []
        SAC_no = []
        a_person = []
        c_person = []
        price_t = []
        rsv_code = []
        ir_code = []
    
        if any(df_station['Station name'].isin([source.title()])) or any(df_station['Station code'].isin([source.upper()])) \
                and any(df_station['Station name'].isin([destination.title()])) or any(df_station['Station code'].isin([destination.upper()])):
            
            source_name = df_station[(df_station['Station name'].isin([source.title()]) | df_station['Station code'].isin([source.upper()]))]['Station name'].values[0]
            destination_name = df_station[(df_station['Station name'].isin([destination.title()]) | df_station['Station code'].isin([destination.upper()]))]['Station name'].values[0]
    
            distance_km = calculate_distance(source_name, destination_name)
    
            if distance_km is not None:
                # print(f"The distance between {source_name} and {destination_name} is {distance_km:.2f} km.")
                Ticket_no = generate_ir_code()
                mo_code = random.randint(100000000000,999999999999)
                sac = random.randint(111111,999999)
                random_code = generate_random_code()
                random_number = generate_random_number()
                adult = int(input('Adult: '))
                children = int(input('Children(6 - 12): '))
                current_time = datetime.now()
                price = fare(distance_km,adult,children)
                formatted_time = current_time.strftime("%H:%M:%S")
                formatted_date = current_time.strftime('%d/%m/%Y')
                p = round(price)
                km_in = round(distance_km)
                datime = f'{formatted_date} {formatted_time}'
                uts_no.append(Ticket_no)
                mobile_code.append(mo_code) 
                from_station.append(source_name) 
                to_station.append(destination_name) 
                datetm.append(datime) 
                dis_in_km.append(km_in) 
                SAC_no.append(sac) 
                a_person.append(adult) 
                c_person.append(children) 
                price_t.append(p)
                rsv_code.append(random_number)
                ir_code.append(random_code)
                
                ticket_data = pd.DataFrame({'UTS NO.':uts_no,
                             'mobile_code':mobile_code,
                             'Source Station':from_station,
                             'Destination':to_station,
                             'Date & Time':datetm,
                             'Distance': dis_in_km,
                             'SAC': SAC_no,
                             'Adult':a_person,
                             'Children':c_person,
                             'Fare': price_t,
                             'IR':ir_code,
                             'Reserve NO':rsv_code
                                }) 
                # df = pd.DataFrame(data = ticket_data )
                df = pd.concat([df, ticket_data], ignore_index=True)
                print(f'''-------------------------------Ticket------------------------------
    HAPPY JOURNEY
    -------------------------------------------------------------------
                                  JOURNEY           {formatted_date}
    ₹Rs {p}/-                                            {mo_code}      
    UTS NO:{Ticket_no}                                                 
    -------------------------------------------------------------------
    (S) {source_name}.                                                    
    (D) {destination_name}.                                               
    ADULT:{adult} CHILD:{children}                                      
    Class: SECOND(||)                                TRAIN: ORDINARY(O)
    -------------------------------------------------------------------
    (Via)---
    -------------------------------------------------------------------
    SAC:{sac}                IR:{random_code}                          
    ---------------------------------------- --------------------------
    Journey Should Commence within 3 Hour Dept of First 
    Train
    {random_number}                    Distance:{round(distance_km)} Km.
    Booking Time:{formatted_date} {formatted_time}
    ''')
                
            else:
                print("Distance calculation failed.")
        else:
            print('Station Not Found:')
    
    except Exception as e:
            print(f"An error occurred: {e}")
            print("Please retry.")

# booking()


# this funcation this booking platform ticket
def plt_booking():
    try:
        global df_pltf
        uts_no = []
        station = []
        date = []
        time = []
        sac_no = []
        person = []
        prt = []
        rsv_c = []
        ir_c = []
        while 1:
            st = input('Rentry Station Name or Code For Confirm: ')
            if any(df_station['Station name'].isin([st.title()])) or any(df_station['Station code'].isin([st.upper()])):
                station_name = df_station[(df_station['Station name'].isin([st.title()]) | df_station['Station code'].isin([st.upper()]))]['Station name'].values[0]
                p = int(input('Enter the NO. of person: '))
                uts_code = generate_ir_code()
                sac_pin = random.randint(111111,999999)
                ir_code = generate_random_code()
                random_number = generate_random_number()
                current_time = datetime.now()
                formatted_time = current_time.strftime("%H:%M:%S")
                formatted_date = current_time.strftime('%d/%m/%Y')
                price = (15*p)
                uts_no.append(uts_code)
                station.append(station_name)
                date.append(formatted_date)
                time.append(formatted_time)
                sac_no.append(sac_pin)
                person.append(p)
                prt.append(price)
                rsv_c.append(random_number)
                ir_c.append(ir_code)
                plt_dataf = pd.DataFrame({'UTS NO.':uts_no,
                                     'Station':station,
                                     'Date': date,
                                     'Time': time,
                                     'SAC': sac_no,
                                     'No. of person':person,
                                     'Fare': prt,
                                     'IR':ir_c,
                                     'Reserve NO':rsv_c
                                        })
                df_pltf = pd.concat([df_pltf, plt_dataf], ignore_index=True)
                
                print(f'''-------------------------------Ticket------------------------------
    HAPPY JOURNEY
    -------------------------------------------------------------------
                                  PLATFORM          {formatted_date}
    ₹Rs {price}/-                                                  
    UTS NO:{uts_code}                                                 
    -------------------------------------------------------------------
    STATION:{station_name}.                                                                                                 
    No. of Person(s).{p}                                      
    -------------------------------------------------------------------
    SAC:{sac_pin}                      IR:{ir_code}                          
    ---------------------------------------- --------------------------
    Platform Ticket is valid up to 2 hour only 
    from booking Time 
    No Printout  is required.No cancellation is allowed.
    {random_number}                  
    Booking Time:{formatted_date}{formatted_time}
    ''')
                return 
            else: print('Station Not Found:')
    except Exception as e:
            print(f"An error occurred: {e}")
            print("Please retry.")
    return

# this funcation is working on platfrom ticket
def platfrom_station():
    while 1:
        pltf = input('Enter the Station OR Code: ')
        if type(pltf) == str:
            suggested_sources = suggest_places(pltf, df_station)
            if suggested_sources:
                for place in suggested_sources:
                    print(f"Station name: {place[0]},Station code: {place[1]}") 
                    plt_booking()
                    return 
            else:
                print(f"No suggestions found for '{pltf}'")

# this funcation is check the mst booking history and print ticket
def history_for_mst():
    global df_mst
    print(df_mst)
    while 1:
        ticket = input('Print Ticket Y/N:').upper()
        if ticket == 'Y':
            id = input('Enter The Booking ID : ').upper()
            data = df_mst[(df_mst['UTS NO.'] == id)]
            source = data['Source Station'].values
            destination = data['Destination'].values
            mo = data['mobile_code'].values
            uts = data['UTS NO.'].values
            dtime = data['Date'].values
            dis = data['Distance'].values
            a = data['Passenger'].values
            c = data['ID'].values
            f = data['Fare'].values
            i = data['IR'].values
            ag = data['Age'].values
            r_code = data['Reserve NO'].values
            s = data['SAC'].values
            tm = data['Time'].values
            end = data['End Date'].values
            tic_m = (f'''-------------------------------Ticket------------------------------
    HAPPY JOURNEY                               MONTHLY
    -------------------------------------------------------------------
                                  ADULT SEASON          {dtime[0]}
    ₹Rs {f[0]}/-                                            {mo[0]} 
    UTS NO:{uts[0]}                                       MONTHLY
    ID Card Number: {c[0]}
    Passenger Name: {a[0]}
    Age:{ag[0]}Years                                        Between
    -------------------------------------------------------------------
    (S) {source[0]}.                                                    
    (D) {destination[0]}.                                                                                     
    CLASS: SECOND(||)                          TRAIN TYPE: ORDINARY(O)
    -------------------------------------------------------------------
    (via)---
    -------------------------------------------------------------------
    SAC:{s[0]}                         IR:{i[0]}                          
    Validity:FROM:{dtime[0]} TO {end[0]}
    {r_code[0]}                    Distance:{dis[0]} Km.
    Booking Time:{dtime[0]} {tm[0]}
    ''')
            print(tic_m)
            q_mode = input('Generate QR Code Y/N: ').upper()
            if q_mode == 'Y':
                qr_code(tic_m)
            elif q_mode == 'N':
                return 
        elif ticket == 'N': return 
    
# history_for_mst()


def booking_history():
    try:
        print(df)
        ticket = input('Print Ticket Y/N:').upper()
        if ticket == 'Y':
            id = input('Enter The Booking ID: ').upper()
            data = df[(df['UTS NO.'] == id)]
            source = data['Source Station'].values
            destination = data['Destination'].values
            mo = data['mobile_code'].values
            uts = data['UTS NO.'].values
            dtime = data['Date & Time'].values
            dis = data['Distance'].values
            a = data['Adult'].values
            c = data['Children'].values
            f = data['Fare'].values
            i = data['IR'].values
            r_code = data['Reserve NO'].values
            s = data['SAC'].values
            d_value = dtime[0]
            date = d_value[:10]
            tic_n = (f'''-------------------------------Ticket------------------------------
    HAPPY JOURNEY
    -------------------------------------------------------------------
                                  JOURNEY                   {date}
    ₹Rs {f[0]}/-                                            {mo[0]}      
    UTS NO:{uts[0]}                                                 
    -------------------------------------------------------------------
    (S) {source[0]}.                                                    
    (D) {destination[0]}.                                               
    ADULT:{a[0]} CHILD:{c[0]}                                      
    Class: SECOND(||)                                TRAIN: ORDINARY(O)
    -------------------------------------------------------------------
    (Via)---
    -------------------------------------------------------------------
    SAC:{s[0]}                IR:{i[0]}                          
    ---------------------------------------- --------------------------
    Journey Should Commence within 3 Hour Dept of First 
    Train
    {r_code[0]}                            Distance:{round(dis[0])} Km.
    Booking Time:{dtime[0]}
    ''')
            print(tic_n)
            q_mode = input('Are You Print QR Code Y/N: ').upper()
            if q_mode == 'Y':
                qr_code(tic_n)
            elif q_mode == 'N':
                return 
        else: return 
    except Exception as e:
        print(f"An error occurred: {e}")
        print("Please retry.")

# booking_history()
    

# this code is show plt booking history
def plt_history():
    global df_pltf
    print(df_pltf)
    ticket = input('Print Ticket Y/N:').upper()
    if ticket == 'Y':
        id = input('Enter The Booking ID: ').upper()
        data = df_pltf[(df_pltf['UTS NO.'] == id)]
        station_n = data['Station'].values
        uts = data['UTS NO.'].values
        dat = data['Date'].values
        per = data['No. of person'].values
        f = data['Fare'].values
        i = data['IR'].values
        r_code = data['Reserve NO'].values
        s = data['SAC'].values
        time_o = data['Time'].values
        tic_p = (f'''-------------------------------Ticket------------------------------
HAPPY JOURNEY
-------------------------------------------------------------------
                              PLATFORM          {dat[0]}
₹Rs {f[0]}/-                                                  
UTS NO:{uts[0]}                                                 
-------------------------------------------------------------------
STATION:{station_n[0]}.                                                                                                 
No. of Person(s).{per[0]}                                      
-------------------------------------------------------------------
SAC:{s[0]}                      IR:{i[0]}                          
---------------------------------------- --------------------------
Platform Ticket is valid up to 2 hour only 
from booking Time 
No Printout  is required.No cancellation is allowed.
{r_code[0]}                  
Booking Time:{dat[0]}{time_o[0]}
''')
        print(tic_p)
        q_mode = input('Are You Print QR Code Y/N: ').upper()
        if q_mode == 'Y':
            qr_code(tic_p)
        elif q_mode == 'N':
            return 
    
    else: return 
    

# plt_history()
    

def booking_history_detail():
    while 1:
        print('''
    1 for Check Normal Ticket Booking History
    2 For Check Platform Ticket Booking History
    3 For Check Season Booking History 
    0 For Exit''')
        mode_in = int(input('Enter Your Option: '))
        if mode_in == 1:
            booking_history()
        elif mode_in == 2:
            plt_history()
        elif mode_in == 3:
            history_for_mst()
        elif mode_in ==0:
            return 

def main_code():
    while 1:
        print('''1 for Booking
2 for platfrom Booking
3 for Session Booking 
4 for show Ticket
0 for Logout''')
        mode = int(input('Enter the any Option: '))
        if mode == 1:
            booking()
        elif mode == 2:
            platfrom_station()
        elif mode == 3:
            booking_mst()
        elif mode == 4:
            booking_history_detail()
        elif mode == 0:
            print('Thank You For Visit')
            break
main_code()       


